In [1]:
!pip install statsmodels==0.12.2

     |████████████████████████████████| 9.5MB 5.4MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.base.transform import BoxCox
from scipy.special import inv_boxcox
from statsmodels.tsa.stattools import kpss
from statsmodels.tools.sm_exceptions import InterpolationWarning
from sklearn.metrics import mean_squared_error as mse
from statsmodels.tsa.stattools import pacf

import warnings
warnings.filterwarnings('ignore', category=InterpolationWarning)

In [3]:
aus_retail = pd.read_csv('https://raw.githubusercontent.com/vpozdnyakov/probabilistic_forecasting/main/datasets/aus_retail.csv',  index_col=0)
aus_retail.index = pd.to_datetime(aus_retail.index)

In [4]:
train_bound = pd.Timestamp('2016-02-01')
train = aus_retail[aus_retail.index <= train_bound].iloc[:, :4]
test = aus_retail[aus_retail.index > train_bound].iloc[:, :4]
train.shape, test.shape

((407, 4), (60, 4))

In [6]:
test_corr = np.corrcoef(test.values.T)
train_corr = np.corrcoef(train.values.T)

print('Correlation loss: {:.4f}'.format(mse(train_corr, test_corr)))

Correlation loss: 0.0035


In [8]:
nts = 4
nlags = 20

test_pacfs = np.zeros((nts, nlags))
for ts_id in range(nts):
    test_pacfs[ts_id] = pacf(test.values[:, ts_id], nlags=nlags)[1:]

train_pacfs = np.zeros((nts, nlags))
for ts_id in range(nts):
    train_pacfs[ts_id] = pacf(train.values[:, ts_id], nlags=nlags)[1:]

print('PACF loss: {:.4f}'.format(mse(train_pacfs, test_pacfs)))

PACF loss: 0.0329
